# Generate SATURN integraion

In [ ]:
# Make the csv
import pandas as pd

df = pd.DataFrame(columns=["path", "species", "embedding_path"])
df["species"] = ["human", "mouse", "lemur"]
df["path"] = ["Vignettes/tabula_mammals/data/human_ct350_tissueTrue_10xTrue.h5ad", "Vignettes/tabula_mammals/data/muris_ct350_tissueTrue.h5ad", 
              "Vignettes/tabula_mammals/data/mouse_lemur_ct350_tissueTrue_10xTrue.h5ad"]

##### CHANGE THESE PATHS #####
human_embedding_path = "/dfs/project/cross-species/yanay/data/proteome/embeddings/Homo_sapiens.GRCh38.gene_symbol_to_embedding_ESM2.pt"
mouse_embedding_path = "/dfs/project/cross-species/yanay/data/proteome/embeddings/Mus_musculus.GRCm39.gene_symbol_to_embedding_ESM2.pt"
lemur_embedding_path = "/dfs/project/cross-species/yanay/data/proteome/embeddings/Microcebus_murinus.Mmur_3.0.gene_symbol_to_embedding_ESM2.pt"
##############################
df["embedding_path"] = [human_embedding_path, mouse_embedding_path, lemur_embedding_path]
df.to_csv("data/tabula_350_run.csv", index=False)
df

In [ ]:
!cd ../../ ; python train-saturn.py --in_data=Vignettes/tabula_mammals/data/tabula_350_run.csv --device_num=6 \
--in_label_col=cell_type --ref_label_col=coarse_cell_type --non_species_batch_col=tissue_type\
--work_dir=Vignettes/tabula_mammals/data/ \
--centroids_init_path=Vignettes/tabula_mammals/data/tabula_final_centroids_6k_15h_350.pkl \
--num_macrogenes=1500 --pretrain --hv_genes=6000 --pretrain_epochs=50 --epochs=10 --model_dim=32 --hidden_dim=256

# Generate the UMAP

In [ ]:
fpath = "./data/metric_results/test16_data_human_ct350_tissueTrue_muris_ct350_tissueTrue_mouse_lemur_ct350_tissueTrue_org_centroids_seed_0.h5ad"

In [ ]:
adata = sc.read(fpath)
display(adata)
if "X_umap" not in adata.obsm:
    sc.pp.pca(adata)
    sc.pl.pca(adata, color="species", size=5)
    sc.pl.pca(adata, color="labels2", size=5)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, min_dist=0.35)
    adata.write(fpath)
    display(adata)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(sharex=False, sharey=False, figsize=(8, 6), frameon=False, dpi=300)
sc.pl.umap(adata, color="species", ax=ax)
fig, ax = plt.subplots(sharex=False, sharey=False, figsize=(8, 6), frameon=False, dpi=300)
sc.pl.umap(adata, color="labels2", ax=ax)
fig, ax = plt.subplots(sharex=False, sharey=False, figsize=(8, 6), frameon=False, dpi=300)
sc.pl.umap(adata, color="ref_labels", ax=ax)
fig, ax = plt.subplots(sharex=False, sharey=False, figsize=(8, 6), frameon=False, dpi=300)
sc.pl.umap(adata, color="batch_labels", ax=ax)

# Make Figure 1B

In [ ]:
available = ["stem cell", 
"barrier cell", 
"immune cell", 
"secretory cell", 
"mesencyhmal",
"hematopoietic"]

map_ref_very_coarse = {
    "T cell":"Immune",
    'alpha-beta T cell':"Immune",
    "phagocyte":"Immune",
    "connective tissue cell":"Mesenchymal",
    "barrier cell":"Barrier",
    "contractile cell":"Mesenchymal",
    "stem cell":"Stem Cell",
    "leukocyte":"Immune",
    "nongranular leukocyte":"Immune",
    "hematopoietic cell":"Hematopoietic",
    "epithelial cell":"Barrier",
    "ciliated cell":"Barrier",
    "secretory cell":"Secretory",
    "hematopoietic precursor cell":"Hematopoietic",
    "lower urinary tract cell":"Barrier",
    "mesenchymal cell":"Mesenchymal",
    "lymphocyte":"Immune",
    "ciliated epithelial cell":"Mesenchymal",
    "cell of skeletal muscle":"Mesenchymal",
    "kidney cell":"Barrier",   
    
}

In [ ]:
mod = 2

In [ ]:
sc._settings.settings._vector_friendly=True

In [ ]:
possible_new_labels = ["Myeloid", "Fibroblast", "Endothelial", "Epithelial",
                      "T/NK cell", "Blood", "B cell", "Neutrophil", "Connective Tissue",
                      "Neuroendocrine", "Epithelial", "Erythroid", "Plasma", "Fibroblast/Mesenchymal"
                      "Skeletal\nMuscle"] 


fibr = "Fibroblast,\nMesenchymal"

tnk = "NK/T Cell"
labels_to_coarser = {
    "macrophage":"Myeloid",
    "fibroblast":fibr,
    "endothelial cell":"Endothelial",
    "smooth muscle cell":"Muscle",
    "t cell":tnk,
    "basal cell":"Epithelial",
    "mast cell":"Myeloid",
    "plasma cell":"Plasma",
    "cd8-positive, alpha-beta t cell":tnk,
    "b cell":"B cell",
    "cd4-positive, alpha-beta t cell":tnk,
    "neutrophil":"Granulocyte",
    "cd8-positive, alpha-beta memory t cell":tnk,
    "naive thymus-derived cd4-positive, alpha-beta t cell":tnk,
    "intermediate monocyte":"Myeloid",
    "memory b cell":"B cell",
    "classical monocyte":"Myeloid",
    "naive b cell":"B cell",
    "cd4-positive, alpha-beta memory t cell":tnk,
    "type i nk t cell":tnk,
    "naive thymus-derived cd8-positive, alpha-beta t cell":tnk,
    "nk cell":tnk,
    "innate lymphoid cell":tnk,
    "erythrocyte":"Erythroid",
    "hematopoietic stem cell":"Hematopoietic",
    "monocyte":"Myeloid",
    "erythroid progenitor":"Hematopoietic",
    "granulocyte":"Granulocyte",
    "nampt neutrophil":"Granulocyte",
    "cd24 neutrophil":"Granulocyte",
    "pancreatic acinar cell":"Epithelial", ###
    "myeloid cell":"Myeloid",
    "pancreatic stellate cell":fibr, ###
    "pancreatic ductal cell":"Epithelial", ###
    "cd4-positive helper t cell":tnk,
    "naive regulatory t cell":tnk,
    "t follicular helper cell":tnk,
    "cd8-positive, alpha-beta cytotoxic t cell":tnk,
    "vascular associated smooth muscle cell":"Muscle",
    "vein endothelial cell":"Endothelial",
    "capillary endothelial cell":"Endothelial",
    "endothelial cell of artery":"Endothelial",
    "endothelial cell of lymphatic vessel":"Endothelial",
    "myofibroblast cell":fibr,
    "bladder urothelial cell":"Epithelial", ###
    "pericyte cell":"Endothelial",
    "type ii pneumocyte":"Type II\nPneumocyte",
    "dn3 thymocyte":tnk,
    "dn1 thymic pro-t cell":tnk,
    "mesenchymal stem cell":fibr,
    "skeletal muscle satellite stem cell":"Skeletal\nMuscle",
    "endothelial cell of vascular tree":"Endothelial",
    "mature nk t cell":tnk,
    "kidney epithelial cell":"Epithelial",
    "basophil":"Myeloid",
    "lung ciliated cell":"Epithelial",
    "respiratory goblet cell":"Epithelial",
    "non-classical monocyte":"Myeloid",
    "capillary aerocyte":"Endothelial",
    "club cell":"Epithelial",
    "lung microvascular endothelial cell":"Endothelial",
    "thymocyte":tnk,
    "kidney capillary endothelial cell":"Endothelial",
    "granulocytopoietic cell":"Myeloid",
    "bladder cell":"",  #### Not sure what this is, don't label it
    "mesenchymal cell":fibr,
    "stromal cell":fibr,
    "kidney loop of henle ascending limb epithelial cell":"Epithelial",
    "hematopoietic precursor cell":"Hematopoietic",
    "natural killer cell":tnk,
    "kidney proximal straight tubule epithelial cell":"Epithelial",
    "epithelial cell":"Epithelial",
    "lung endothelial cell":"Endothelial",
    "kidney collecting duct epithelial cell":"Epithelial",
    "neuroendocrine cell":"Neuroendocrine",
    "immature t cell":tnk,
    "blood cell":"Myeloid",
    "skeletal muscle satellite cell":"Skeletal\nMuscle",
    "mesothelial cell":"Epithelial",
    "dendritic cell":"Myeloid",
    "lymphocyte":tnk,
    "alveolar macrophage":"Myeloid",
    "fibroblast of lung":fibr,
    "vasa recta ascending limb cell":"Endothelial",
    "kidney loop of henle thin ascending limb epithelial cell":"Epithelial",
    "kidney loop of henle thin descending limb epithelial cell":"Epithelial",
    "vasa recta descending limb cell":"Endothelial",
    "kidney proximal convoluted tubule epithelial cell":"Epithelial",
    "renal alpha-intercalated cell":"Epithelial",
    "erythroid progenitor cell":"Hematopoietic",
    "kidney loop of henle thick ascending limb epithelial cell":"Epithelial",
    "erythroid lineage cell":"Erythroid",
    
    
    # 150 CT
    "ciliated cell":"Epithelial",
    "secretory cell":"Epithelial",
    "regulatory t cell":tnk,
    "myeloid progenitor":"Myeloid",
    "medullary thymic epithelial cell":"Epithelial",
    "fast muscle cell":"Muscle",
    "tendon cell":"Muscle",
    "alveolar fibroblast":fibr,
    "connective tissue cell":fibr,
    "leukocyte":"Myeloid",
    "proerythroblast":"Erythroid",
    "erythroblast":"Erythroid",
    "promonocyte":"Myeloid",
    "late pro-b cell":"B cell",
    "fat cell":fibr, # 
    "conventional dendritic cell":"Myeloid",
    "plasmacytoid dendritic cell":"Myeloid",
    "type i pneumocyte":"Type I\nPneumocyte",
    "renal principal cell":"Epithelial",
    "epithelial cell of proximal tubule":"Epithelial",
    "glomerular endothelial cell":"Endothelial",
}

In [ ]:
alignment_column = "labels2"

In [ ]:
adata.obs["coarser_labels"] = [labels_to_coarser.get(l.lower(), "misc") for l in adata.obs[alignment_column]]

In [ ]:
non_mapped = adata.obs[alignment_column].unique()
[print(f'"{l.lower()}":"",') for l in non_mapped if l.lower() not in labels_to_coarser.keys()];

In [ ]:
!mkdir figures

In [ ]:
fig, ax = plt.subplots(sharex=False, sharey=False, figsize=(4, 5), frameon=False, dpi=300)

ax = sc.pl.umap(adata, color="coarser_labels", show=False, alpha=1, ax=ax, legend_fontsize=7, legend_fontoutline=2, legend_loc="on data")
ax.set(xlabel=None, ylabel=None);
ax.set(title=None)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.savefig("figures/1b_main.svg")    
plt.show()


In [ ]:
fig, ax = plt.subplots(sharex=False, sharey=False, figsize=(4, 5), frameon=False, dpi=300)

ax = sc.pl.umap(adata, color="species", show=False, alpha=1, ax=ax, legend_loc=None)
ax.set(xlabel=None, ylabel=None);
ax.set(title=None)
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.savefig("figures/1b_spec.svg")
plt.show()
xlims = ax.get_xlim()
ylims = ax.get_ylim()

In [ ]:
fig, ax = plt.subplots(sharex=False, sharey=False, figsize=(6, 4), frameon=False, dpi=300)

ax = sc.pl.umap(adata, color="labels2", show=False, alpha=1, ax=ax, legend_loc=None,legend_fontsize=8, legend_fontoutline=2)
ax.set(xlabel=None, ylabel=None);
ax.set(title=None)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.savefig("figures/1b_type.svg")
plt.show()
xlims = ax.get_xlim()
ylims = ax.get_ylim()